In [1]:
import torch
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

###Dataset

In [3]:
random_seed = 123
learning_rate = 0.1
num_epochs = 10
batch_size = 256

num_features = 784
num_classes = 10

train_dataset = datasets.MNIST(root='data',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_datasets = datasets.MNIST(root='data',
                               train=False,
                               transform=transforms.ToTensor(),
                               download=True)

train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=True)

test_loader = DataLoader(dataset=test_datasets,
                         batch_size=batch_size,
                         shuffle=False)

for images,labels in train_loader:
  print('Image batch dimension: ', images.shape)
  print('Image label dimensions: ', labels.shape)
  break

100%|██████████| 9.91M/9.91M [00:00<00:00, 34.3MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.11MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 8.32MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 4.87MB/s]

Image batch dimension:  torch.Size([256, 1, 28, 28])
Image label dimensions:  torch.Size([256])


###Model

In [14]:
class SoftmaxRegression(torch.nn.Module):

  def __init__(self,num_features,num_classes):
    super(SoftmaxRegression,self).__init__()
    self.linear = torch.nn.Linear(num_features,num_classes)

  def forward(self,x):
    logits = self.linear(x)
    probas = F.softmax(logits,dim=1)
    return logits,probas

model = SoftmaxRegression(num_features = num_features,num_classes=num_classes)

model.to(device)

optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate)



In [15]:
torch.manual_seed(random_seed)

def compute_accuracy(model,data_loader):
  correct_pred, num_examples = 0,0

  for features,targets in data_loader:
    features = features.view(-1,28*28).to(device)
    targets = targets.to(device)
    logits,probas = model(features)
    _,predicted_labels = torch.max(probas,1)
    num_examples += targets.size(0)
    correct_pred += (predicted_labels == targets).sum()

  return correct_pred.float()/num_examples*100

for epoch in range(num_epochs):
  for batch_idx, (features,targets) in enumerate(train_loader):

    features = features.view(-1,28*28).to(device)
    targets = targets.to(device)

    logits,probas = model(features)

    cost = F.cross_entropy(logits,targets)
    optimizer.zero_grad()
    cost.backward()

    optimizer.step()

    if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f'
                   %(epoch+1, num_epochs, batch_idx,
                     len(train_dataset)//batch_size, cost))

  with torch.set_grad_enabled(False):
        print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
              epoch+1, num_epochs,
              compute_accuracy(model, train_loader)))

Epoch: 001/010 | Batch 000/234 | Cost: 2.3151
Epoch: 001/010 | Batch 050/234 | Cost: 0.7719
Epoch: 001/010 | Batch 100/234 | Cost: 0.5332
Epoch: 001/010 | Batch 150/234 | Cost: 0.5600
Epoch: 001/010 | Batch 200/234 | Cost: 0.5498
Epoch: 001/010 training accuracy: 88.03%
Epoch: 002/010 | Batch 000/234 | Cost: 0.4817
Epoch: 002/010 | Batch 050/234 | Cost: 0.3706
Epoch: 002/010 | Batch 100/234 | Cost: 0.4558
Epoch: 002/010 | Batch 150/234 | Cost: 0.3754
Epoch: 002/010 | Batch 200/234 | Cost: 0.4780
Epoch: 002/010 training accuracy: 89.22%
Epoch: 003/010 | Batch 000/234 | Cost: 0.4066
Epoch: 003/010 | Batch 050/234 | Cost: 0.4468
Epoch: 003/010 | Batch 100/234 | Cost: 0.3790
Epoch: 003/010 | Batch 150/234 | Cost: 0.3870
Epoch: 003/010 | Batch 200/234 | Cost: 0.3011
Epoch: 003/010 training accuracy: 89.96%
Epoch: 004/010 | Batch 000/234 | Cost: 0.4274
Epoch: 004/010 | Batch 050/234 | Cost: 0.3931
Epoch: 004/010 | Batch 100/234 | Cost: 0.3769
Epoch: 004/010 | Batch 150/234 | Cost: 0.3922
Epo

In [16]:
accuracy = compute_accuracy(model,test_loader)
print(f'Test accuracy: {accuracy:.2f}%')

Test accuracy: 91.65%
